# Implementation of NCSN (Noise Conditional Score Networks)

In [1]:
%load_ext autoreload
%autoreload 2
!nvidia-smi
!which python

Tue Oct 22 19:49:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000004:04:00.0 Off |                    0 |
| N/A   44C    P0              43W / 300W |      0MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
try:
    from NCSN.train import train, make_dataset, cal_noise_level, langevin
    from NCSN.utils import train_set, val_set
    from NCSN.model import CondUNet, UNetv2
except:
    from train import train, make_dataset, cal_noise_level, langevin
    from utils import train_set, val_set
    from model import CondUNet, UNetv2

from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import datetime
import os

train_loader = DataLoader(train_set, 256, shuffle=True, drop_last=False, pin_memory=True)
val_loader = DataLoader(val_set, 500, shuffle=True, drop_last=False, pin_memory=True)

def timestr():
    now = datetime.datetime.now()
    return now.strftime("%Y%m%d_%H%M%S")

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Here are the hyperparameters

epochs = 200
criterion = nn.MSELoss()
init_sigma = 28
final_sigma = 0.01
n_sigma = 75
eps = 1e-5
T = 5
eval_freq = 5
ema_decay = 0.999

sigmas = cal_noise_level(init_sigma, final_sigma, n_sigma)
model = UNetv2(sigmas=torch.tensor(sigmas))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

print("model:", model)
# calculate the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print("number of parameters:", num_params)


time_str = timestr()
print("time string:", time_str)

train(epochs=epochs, model=model, optimizer=optimizer, criterion=criterion, train_loader=train_loader, val_loader=val_loader, sigmas=sigmas, eps=eps, T=T, time_str=time_str, eval_freq=eval_freq, ema_decay=ema_decay)

model: UNetv2(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_up1): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_up2): Conv2d(32, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_up3): Conv2d(8, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (up1): ConvTranspose2d(256, 64, kernel_size=(2, 2), stride=(2, 2))
  (up2): ConvTranspose2d(32, 16, kernel_size=(2, 2), stride=(2, 2))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
)
number of parameters: 264097
time string: 20241022_182843
code files copied


Epoch 2/200:   1%|▍                                                     | 2/235 [00:00<00:20, 11.39it/s, Train Loss=1.5]

Epoch 0: train loss 0.859027462817253


Epoch 3/200:   1%|▍                                                    | 2/235 [00:00<00:20, 11.39it/s, Train Loss=1.24]

Epoch 1: train loss 0.6839413024009542


...

Epoch 197: train loss 0.10818495563370116


Epoch 200/200:   1%|▍                                                 | 2/235 [00:00<00:20, 11.38it/s, Train Loss=0.222]

Epoch 198: train loss 0.10855060993356908


Epoch 199: train loss 0.10697510635599176


Corruption MSE: 0.063483, Recovered MSE: 0.014693: 100%|███████████████████████████| 2000/2000 [00:07<00:00, 270.44it/s]


In [4]:
# # load a model
# import os
# model = UNet(L=n_sigma).cuda()
# print(sum(p.numel() for p in model.parameters() if p.requires_grad))
# eval_epoch = 70
# model.load_state_dict(torch.load(f'NCSN/models/{eval_epoch}.pth'))
# make_dataset(model, sigmas, eps=eps, T=T)
# print("Dataset created")
# os.system("python NCSN/evaluate.py")

In [1]:
# sampling
try:
    from NCSN.utils import val_set
    from NCSN.model import CondUNet, UNetv2
    from NCSN.train import evaluate_denoising, cal_noise_level, langevin
    from NCSN.ema import EMAHelper

except:
    from utils import val_set
    from model import CondUNet, UNetv2
    from train import evaluate_denoising, cal_noise_level, langevin
    from ema import EMAHelper

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# val_loader = DataLoader(val_set, 500, shuffle=True, drop_last=False, pin_memory=True)

criterion = nn.MSELoss()
init_sigma = 28
final_sigma = 0.01
n_sigma = 75
eps = 6e-6
T = 10
ema_decay = 0.999

sigmas = cal_noise_level(init_sigma, final_sigma, n_sigma)

model = UNetv2(sigmas=torch.tensor(sigmas)).cuda()
time_str = "20241022_182843"
eval_epoch = 199

if ema_decay is not None:
    ema_helper = EMAHelper(ema_decay)
    ema_helper.register(model)
    ema_helper.load_state_dict(torch.load(f"/nobackup/users/sqa24/NCSN/{time_str}/models/{eval_epoch:03d}_ema.pth"))
    ema_helper.ema(model)
else:
    model.load_state_dict(torch.load(f"/nobackup/users/sqa24/NCSN/{time_str}/models/{eval_epoch:03d}.pth"))
model.cuda()

# x = torch.rand(10, 1, 28, 28).cuda()
x = torch.randn(10, 1, 28, 28).cuda()
x = (x+1)/2
y = langevin(model, x, sigmas, eps=eps, T=T, save=True, epochs=eval_epoch, clamp=False, time_str=time_str)

Files already downloaded and verified
Files already downloaded and verified


In [1]:
# visualize denoising
try:
    from NCSN.utils import val_set
    from NCSN.model import CondUNet, UNetv2
    from NCSN.train import evaluate_denoising, cal_noise_level, langevin
    from NCSN.ema import EMAHelper

except:
    from utils import val_set
    from model import CondUNet, UNetv2
    from train import evaluate_denoising, cal_noise_level, langevin
    from ema import EMAHelper

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

val_loader = DataLoader(val_set, 10, shuffle=True, drop_last=False, pin_memory=True)

criterion = nn.MSELoss()
init_sigma = 28
final_sigma = 0.01
n_sigma = 75
eps = 6e-6
T = 10
ema_decay = 0.999

sigmas = cal_noise_level(init_sigma, final_sigma, n_sigma)

model = UNetv2(sigmas=torch.tensor(sigmas)).cuda()
time_str = "20241022_182843"
eval_epoch = 199

if ema_decay is not None:
    ema_helper = EMAHelper(ema_decay)
    ema_helper.register(model)
    ema_helper.load_state_dict(torch.load(f"/nobackup/users/sqa24/NCSN/{time_str}/models/{eval_epoch:03d}_ema.pth"))
    ema_helper.ema(model)
else:
    model.load_state_dict(torch.load(f"/nobackup/users/sqa24/NCSN/{time_str}/models/{eval_epoch:03d}.pth"))
model.cuda()

corruption_mse, mse, original, broken, recovered = evaluate_denoising(
    model, sigmas, eps=eps, T=T, val_loader=val_loader, outdir=f'NCSN/rubbish.out', visualize=True
)

Files already downloaded and verified
Files already downloaded and verified


Corruption MSE: 0.061190, Recovered MSE: 0.008740:   0%|▏                             | 10/2000 [00:01<05:54,  5.61it/s]


In [2]:
# experiment on recovering
try:
    from NCSN.utils import val_set
    from NCSN.model import CondUNet, UNetv2
    from NCSN.train import evaluate_denoising, cal_noise_level, langevin
    from NCSN.ema import EMAHelper

except:
    from utils import val_set
    from model import CondUNet, UNetv2
    from train import evaluate_denoising, cal_noise_level, langevin
    from ema import EMAHelper

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

val_loader = DataLoader(val_set, 500, shuffle=True, drop_last=False, pin_memory=True)

criterion = nn.MSELoss()
init_sigma = 28
final_sigma = 0.01
n_sigma = 75
eps = 6e-6
T = 10
ema_decay = 0.999

sigmas = cal_noise_level(init_sigma, final_sigma, n_sigma)

model = UNetv2(sigmas=torch.tensor(sigmas)).cuda()
time_str = "20241009_193254"
eval_epoch = 10

if ema_decay is not None:
    ema_helper = EMAHelper(ema_decay)
    ema_helper.register(model)
    ema_helper.load_state_dict(torch.load(f"/nobackup/users/sqa24/NCSN/{time_str}/models/{eval_epoch:03d}_ema.pth"))
    ema_helper.ema(model)
else:
    model.load_state_dict(torch.load(f"/nobackup/users/sqa24/NCSN/{time_str}/models/{eval_epoch:03d}.pth"))
model.cuda()

corruption_mse, mse, original, broken, recovered = evaluate_denoising(
    model, sigmas, eps=eps, T=T, val_loader=val_loader, outdir=f'NCSN/rubbish.out'
)

FileNotFoundError: [Errno 2] No such file or directory: '/nobackup/users/sqa24/NCSN/20241009_193254/models/010_ema.pth'